<a href="https://colab.research.google.com/github/DishaJillella/Django-polls/blob/master/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your rag.txt here

Saving rag.txt to rag.txt


In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.5 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle

model = SentenceTransformer('all-MiniLM-L6-v2')

doc_texts = []

# Read the uploaded file
with open("rag.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chunks = text.split('.')  # Simple split by period
    for chunk in chunks:
        if chunk.strip():
            doc_texts.append(chunk.strip())

# Create embeddings
embeddings = model.encode(doc_texts)

# Store in FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Save index and chunks
with open("embeddings.pkl", "wb") as f:
    pickle.dump(doc_texts, f)
faiss.write_index(index, "index.faiss")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import pickle
import faiss

# Load everything
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.read_index("index.faiss")
with open("embeddings.pkl", "rb") as f:
    doc_texts = pickle.load(f)
generator = pipeline("text-generation", model="gpt2")

# Ask your question!
question = input("Ask your question: ")
q_embedding = retriever_model.encode([question])
D, I = index.search(q_embedding, k=3)

# Get relevant text
retrieved_text = " ".join([doc_texts[i] for i in I[0]])

# Generate answer
input_text = f"Context: {retrieved_text}\nQuestion: {question}\nAnswer:"
output = generator(input_text, max_length=150, do_sample=True)
print("\n📌 Answer:\n", output[0]['generated_text'])


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


Ask your question: What is django


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📌 Answer:
 Context: Django is a high-level Python web framework that promotes rapid development and clean, pragmatic design A Django project is made up of applications To run your Django project, you use `python manage
Question: What is django
Answer: Django is a high-level Python web framework that promotes rapid development and clean, pragmatic design. A Django project is made up of applications. Django is a framework that includes a web server and a web API for development and testing. It's a simple approach to managing and developing your Django applications.
Question: What's new
Question: What's new
Answer: Django is a high-level Python web framework that promotes rapid development and clean, pragmatic design. The app architecture is a simple set of modules that allow you to use most of the power of Django.
Question: What's new
Question: What's new
Answer: Django is a high-level Python web framework that promotes rapid development and clean, pragmatic design. Django is a framewor